# Lesson 3 - Calling an A2A Agent using an A2A Client

In this lesson, you will use the A2A Python Client to interact with the Policy Agent you created and activated in the previous lesson. The client handles the protocol details, allowing you to discover the agent's capabilities via its `AgentCard` and send messages to it.

In [ ]:
# !uv pip show a2a-sdk


Name: a2a-sdk
Version: 0.3.16
Location: c:\Users\loint\_repos\a2a\A2AWalkthrough\.venv\Lib\site-packages
Requires: google-api-core, httpx, httpx-sse, protobuf, pydantic
Required-by: agent-framework-a2a, langgraph-a2a-server


In [8]:
import os

import httpx
from IPython.display import Markdown, display
from a2a.client import (
    Client,
    ClientConfig,
    ClientFactory,
    create_text_message_object,
)
from a2a.types import Artifact, Message, Task
from a2a.utils.message import get_message_text

from helpers import display_agent_card, setup_env

## 3.1. Check Server Status

First, ensure that your `PolicyAgent` server is running.
- Open a terminal as instructed below.
- If the agent is still running from the previous lesson, you don't need to do anything.
- If the agent has stopped, type: `uv run a2a_policy_agent.py`

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open a new terminal window in your Jupyter environment to run the server.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 3.2. Setup Client Configuration

Load environment variables to get the host and port of the running agent. You will construct the connection URL from these variables.


In [9]:
setup_env()

host = os.getenv("AGENT_HOST", "localhost")
port = os.getenv("POLICY_AGENT_PORT")
# port = os.getenv("RESEARCH_AGENT_PORT")

In [10]:
prompt = "How much would I pay for mental health therapy?"
# prompt = "How do I get mental health therapy?"

## 3.3. Run the Client Interaction

Now you will execute the client interaction flow:
1.  **Connect**: Establish a connection to the agent using `ClientFactory`.
2.  **Discover**: Retrieve the `AgentCard` using `get_card()`.
3.  **Construct**: Create a message object with the user's prompt.
4.  **Send & Receive**: Send the message and asynchronously process the response stream (which may include Tasks, Artifacts, or Messages).

 
Your Notebook
     │
     │  1. Connect (like opening a phone line)
     ▼
Agent Server  ──►  2. Get Agent Card (read its "business card")
     │
     │  3. Send your question as a structured Message
     ▼
Agent processes it...
     │
     │  4. Stream back responses (either a Message or a Task+Artifact)
     ▼
Your Notebook  ──►  5. Display the final answer 

Key Concepts for Beginners

Term	        What it means
async/await	    The code waits for slow operations (network calls) without freezing
Agent Card	    The agent's self-description (name, skills, version)
Message	         A simple, direct reply from the agent
Task	              A more complex job the agent ran (may produce Artifacts)
Artifact          A file or piece of output produced by a Task
async for	          Loop over a stream of events as they arrive



In [ ]:
# httpx is a modern HTTP library for Python.
# We use "async with" so the HTTP connection is automatically closed when done.
# timeout=100.0 means: wait up to 100 seconds for the agent to respond before giving up.
async with httpx.AsyncClient(timeout=100.0) as httpx_client:

    # ─────────────────────────────────────────────
    # STEP 1: Connect to the remote A2A Agent
    # ─────────────────────────────────────────────
    # ClientFactory.connect() establishes a connection to the agent server
    # running at the given address (e.g. http://localhost:8000).
    # Think of this like "dialing" the agent — you're not talking yet, just connecting.
    client: Client = await ClientFactory.connect(
        f"http://{host}:{port}",          # The agent's server address
        client_config=ClientConfig(
            httpx_client=httpx_client,    # Reuse the HTTP connection we opened above
        ),
    )

    # ─────────────────────────────────────────────
    # STEP 2: Discover what the agent can do
    # ─────────────────────────────────────────────
    # Every A2A agent exposes an "Agent Card" — a JSON description of itself.
    # It contains: the agent's name, description, supported skills, version, etc.
    # This is like reading the agent's "business card" before talking to it.
    agent_card = await client.get_card()
    display_agent_card(agent_card)  # Pretty-print the card so you can see agent info

    # ─────────────────────────────────────────────
    # STEP 3: Prepare the message to send
    # ─────────────────────────────────────────────
    # The A2A protocol has a specific message format (not just a plain string).
    # create_text_message_object() wraps your plain text prompt into that format.
    # Example: "What is our refund policy?" → becomes a structured A2A Message object
    message = create_text_message_object(content=prompt)

    display(Markdown(f"**Sending prompt:** `{prompt}` to the agent..."))

    # ─────────────────────────────────────────────
    # STEP 4: Send the message to the agent
    # ─────────────────────────────────────────────
    # client.send_message() sends the message and returns an async generator.
    # NOTE: The agent hasn't responded YET at this point!
    # "responses" is like a stream — we'll receive replies one by one in Step 5.
    responses = client.send_message(message)

    text_content = ""  # This will hold the final text answer from the agent

    # ─────────────────────────────────────────────
    # STEP 5: Read the agent's response(s)
    # ─────────────────────────────────────────────
    # "async for" means: keep reading replies from the stream until there are no more.
    # The agent might send back multiple events (updates, status changes, final answer).
    async for response in responses:

        # ── Case A: The agent replied directly with a final Message ──
        # This happens when the agent answers immediately (simple, short tasks).
        if isinstance(response, Message):
            print(f"Message ID: {response.message_id}")
            # get_message_text() extracts the plain text from the A2A Message object
            text_content = get_message_text(response)

        # ── Case B: The agent returned a Task (long-running work) ──
        # This happens when the agent ran a multi-step process (e.g., looked up a DB,
        # called other agents, or generated a file/artifact).
        # "tuple" here means (Task, ClientEvent) packed together.
        elif isinstance(response, tuple):
            task: Task = response[0]       # The Task object containing the result
            print(f"Task ID: {task.id}")

            # An "Artifact" is a piece of output produced by the task
            # (e.g., a document, a report, or a data file).
            if task.artifacts:
                artifact: Artifact = task.artifacts[0]  # Take the first artifact
                print(f"Artifact ID: {artifact.artifact_id}")
                # Extract the text content from the artifact
                text_content = get_message_text(artifact)

    # ─────────────────────────────────────────────
    # STEP 6: Display the final result
    # ─────────────────────────────────────────────
    display(Markdown("### Final Agent Response\n-----"))

    if text_content:
        # Render the agent's answer as nicely formatted Markdown in the notebook
        display(Markdown(text_content))
    else:
        # If we got no usable text back, show a friendly error message
        display(
            Markdown(
                "**No final text content received or task did not complete successfully.**"
            )
        )


### Agent Card Details
| Property | Value |
| :--- | :--- |
| **Name** | InsurancePolicyCoverageAgent |
| **Description** | Provides information about insurance policy coverage options and details. |
| **Version** | `1.0.0` |
| **URL** | [http://localhost:9999/](http://localhost:9999/) |
| **Protocol Version** | `0.3.0` |

#### Skills
| Name | Description | Examples |
| :--- | :--- | :--- |
| **Insurance coverage** | Provides information about insurance coverage options and details. | • What does my policy cover?<br>• Are mental health services included? |

**Sending prompt:** `How much would I pay for mental health therapy?` to the agent...

Message ID: 0c8f60ec-75c5-4030-b756-703711ef3633


### Final Agent Response
-----

For mental health therapy under the gHIP HDHP plan, your costs depend on whether the provider is in-network and if you have met your deductible:

*   **In-Network Providers:** You pay **10% coinsurance** after your deductible is met.
*   **Out-of-Network Providers:** You pay **30% coinsurance** after your deductible is met.

**Important Deductible Information:**
Because this is a High Deductible Health Plan (HDHP), you generally must pay 100% of the costs out-of-pocket until you reach your **overall deductible**, which is:
*   **\$1,700** for individuals / **\$3,400** for families (In-Network)
*   **\$3,400** for individuals / **\$6,800** for families (Out-of-Network)

Once the deductible is met, the coinsurance rates listed above apply until you reach your out-of-pocket limit.

## 3.4. Resources

- [A2A Python SDK Documentation](https://a2a-protocol.org/latest/sdk/python/api/)
